In [1]:
import sys
sys.path.append("/home/webphy/Desktop/dnn_processor/")  # just to enable `dataset`
sys.path.append("/home/webphy/Desktop/dnn_processor/dataset/")  # just to enable `dataset`

import tensorflow as tf
import dataset
import numpy as np
import keras
from datetime import datetime
from keras import layers

2024-02-10 19:53:15.606483: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-10 19:53:15.630473: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 19:53:15.630493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 19:53:15.631242: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-10 19:53:15.635300: I tensorflow/core/platform/cpu_feature_guar

In [2]:
DEFAULT_BATCH_SIZE = 64
EPOCHS = 100
DEFAULT_INPUT_SIZE = 128

train_images_rgx = "../../dataset/train_set_4x/*/*.jpeg"
test_images_rgx = "../../dataset/test_set_4x/*/*.jpeg"

train_dataset, test_dataset, num_classes = dataset.create_datasets(
    train_images_rgx, test_images_rgx, DEFAULT_BATCH_SIZE, DEFAULT_INPUT_SIZE
)

2024-02-10 19:53:16.529554: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:53:16.552697: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-10 19:53:16.552808: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
EXP_ID = "baseline_model"


model = keras.models.load_model(
    f"../handmade/saved_models/{EXP_ID}/best_model_t/model.h5",
    compile=False,
)
model.compile(
    keras.optimizers.legacy.Adam(), loss=tf.keras.losses.CategoricalCrossentropy(True), metrics=["accuracy"]
)

score = model.evaluate(test_dataset)
print("Accuracy: {}".format(score[1]))

model.summary()

2024-02-10 19:53:18.837430: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


5/5 [==============================] - 2s 21ms/step - loss: 0.4989 - accuracy: 0.8582
Accuracy: 0.8582375645637512
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 4)         196       
                                                                 
 activation (Activation)     (None, 32, 32, 4)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 15, 15, 16)        592       
                                                                 
 activation_1 (Activation)   (None, 15, 15, 16)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 15, 15, 5)         85        
            

# quantize model

In [4]:
import qkeras
from qkeras import *
from keras.layers import (
    Conv2D,
    DepthwiseConv2D,
    Activation,
    Dense,
    Flatten,
    MaxPooling2D,
    GlobalAveragePooling2D,
)


BIT_WIDTH = 8
DEFAULT_ALPHA = 1


def create_qkeras_model_from_keras_model(keras_model):
    x = x_in = keras.Input(shape=keras_model.layers[0].output.shape[1:])

    x = QActivation(quantized_relu(BIT_WIDTH, 0))(x)

    for l in keras_model.layers:
        if isinstance(l, Conv2D):
            qconv2d = QConv2D(
                l.filters,
                l.kernel_size,
                l.strides,
                l.padding,
                kernel_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            x = qconv2d(x)
            qconv2d.set_weights(l.get_weights())

        elif isinstance(l, DepthwiseConv2D):
            qDepthwiseConv2D = QDepthwiseConv2D(
                l.kernel_size,
                l.strides,
                l.padding,
                depthwise_quantizer=quantized_bits(
                    BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA
                ),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            x = qDepthwiseConv2D(x)
            qDepthwiseConv2D.set_weights(l.get_weights())

        elif isinstance(l, Activation):
            activation_name = l.get_config()["activation"]
            print(activation_name)
            if activation_name == "tanh":
                x = QActivation(quantized_tanh(BIT_WIDTH, True))(x)
            elif activation_name == "relu":
                x = QActivation(quantized_relu(BIT_WIDTH, 0))(x)
            elif activation_name == "linear":
                x = QActivation(quantized_bits(BIT_WIDTH, 0, 1))(x)
            else:
                print(f"{l.name} was not quantized")

        elif isinstance(l, Flatten):
            # necessary because the layer before Flatten doesnt have avtivation
            x = QActivation(quantized_bits(BIT_WIDTH - 4, 0, 1))(x)
            x = Flatten()(x)

        elif isinstance(l, Dense):
            qdense = QDense(
                l.units,
                kernel_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
                bias_quantizer=quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA),
            )
            # just for testing the latency
            x = qdense(x)
            qdense.set_weights(l.get_weights())

        elif isinstance(l, MaxPooling2D):
            x = MaxPooling2D()(x)

        elif isinstance(l, GlobalAveragePooling2D):
            globalAveragePooling2D = GlobalAveragePooling2D()
            x = globalAveragePooling2D(x)
            x = QActivation(quantized_bits(BIT_WIDTH, 0, 1, alpha=DEFAULT_ALPHA))(x)
        else:
            print(f"{l.name} layer was not quantized...")

    qkeras_model = keras.models.Model(x_in, x)

    return qkeras_model


qkeras_model = create_qkeras_model_from_keras_model(model)
qkeras_model.compile(
    keras.optimizers.Adam(0.0005),
    loss=keras.losses.CategoricalCrossentropy(True),
    metrics=["accuracy"],
)
qkeras_model.summary()

input_1 layer was not quantized...
relu
relu
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 q_activation (QActivation)  (None, 128, 128, 3)       0         
                                                                 
 q_conv2d (QConv2D)          (None, 32, 32, 4)         196       
                                                                 
 q_activation_1 (QActivatio  (None, 32, 32, 4)         0         
 n)                                                              
                                                                 
 q_conv2d_1 (QConv2D)        (None, 15, 15, 16)        592       
                                                                 
 q_activation_2 (QActivatio  (None, 15, 15, 16)        0         
 n)             

In [5]:
from qkeras.autoqkeras.utils import print_qmodel_summary
print_qmodel_summary(qkeras_model)

q_activation         quantized_relu(8,0)
q_conv2d             f=4 quantized_bits(8,0,1,alpha=1) quantized_bits(8,0,1,alpha=1) 
q_activation_1       quantized_relu(8,0)
q_conv2d_1           f=16 quantized_bits(8,0,1,alpha=1) quantized_bits(8,0,1,alpha=1) 
q_activation_2       quantized_relu(8,0)
q_conv2d_2           f=5 quantized_bits(8,0,1,alpha=1) quantized_bits(8,0,1,alpha=1) 
q_activation_3       quantized_bits(4,0,1)
q_dense              u=5 quantized_bits(8,0,1,alpha=1) quantized_bits(8,0,1,alpha=1) 



In [7]:
from qkeras.estimate import print_qstats

print_qstats(qkeras_model)
qkeras_model.evaluate(test_dataset)
# free memory
del model


Number of operations in model:
    q_conv2d                      : 196608 (smult_8_8)
    q_conv2d_1                    : 129600 (smult_8_8)
    q_conv2d_2                    : 18000 (smult_8_8)
    q_dense                       : 5625  (smult_8_4)

Number of operation types in model:
    smult_8_4                     : 5625
    smult_8_8                     : 344208

Weight profiling:
    q_conv2d_weights               : 192   (8-bit unit)
    q_conv2d_bias                  : 4     (8-bit unit)
    q_conv2d_1_weights             : 576   (8-bit unit)
    q_conv2d_1_bias                : 16    (8-bit unit)
    q_conv2d_2_weights             : 80    (8-bit unit)
    q_conv2d_2_bias                : 5     (8-bit unit)
    q_dense_weights                : 5625  (8-bit unit)
    q_dense_bias                   : 5     (8-bit unit)

Weight sparsity:
... quantizing model
    q_conv2d                       : 0.0408
    q_conv2d_1                     : 0.0118
    q_conv2d_2                     

In [8]:
qkeras_model.fit(train_dataset, batch_size=256, epochs=5, validation_data=test_dataset)
trial_id = "qatwqkeras"
qkeras_model.save(f"qatwqkeras/saved_qmodels/{trial_id}.h5")

Epoch 1/5


2024-02-10 19:59:14.410794: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/model/q_activation_2/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-02-10 19:59:14.528626: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f60ec088220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-10 19:59:14.528647: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2024-02-10 19:59:14.531873: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1707605954.589553  799098 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


55/55 [==============================] - 2s 6ms/step - loss: 0.5569 - accuracy: 0.9489 - val_loss: 0.4938 - val_accuracy: 0.8544
Epoch 2/5
55/55 [==============================] - 0s 6ms/step - loss: 0.0781 - accuracy: 0.9770 - val_loss: 0.4774 - val_accuracy: 0.8621
Epoch 3/5
55/55 [==============================] - 0s 6ms/step - loss: 0.0753 - accuracy: 0.9801 - val_loss: 0.4681 - val_accuracy: 0.8697
Epoch 4/5
55/55 [==============================] - 0s 6ms/step - loss: 0.0736 - accuracy: 0.9801 - val_loss: 0.4781 - val_accuracy: 0.8544
Epoch 5/5
55/55 [==============================] - 0s 6ms/step - loss: 0.0726 - accuracy: 0.9815 - val_loss: 0.4763 - val_accuracy: 0.8736


/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
/home/webphy/Desktop/dnn_processor_v2/.venv/lib/python3.10/site-packages/keras/src/constraints.py:365: UserWarning: The `keras.constraints.serialize()` API should only be used for objects of type `keras.constraints.Constraint`. Found an instance of type <class 'qkeras.quantizers.quantized_bits'>, which may lead to improper serialization.
  warnings.warn(


In [9]:
from qkeras.utils import _add_supported_quantized_objects

co = {}
_add_supported_quantized_objects(co)

loaded_qkeras_model = keras.models.load_model(f"qatwqkeras/saved_qmodels/{trial_id}.h5", co)
loaded_qkeras_model.evaluate(test_dataset)

# free memory
del loaded_qkeras_model

5/5 [==============================] - 0s 4ms/step - loss: 0.4763 - accuracy: 0.8736
